# 📊 Testing BDH's "Scale-Free" Claims

The BDH paper uses "scale" ambiguously. Let's test what they might mean:

| Interpretation | What It Means | How We Test |
|----------------|---------------|-------------|
| Scale-free network | Power-law degree distribution | Analyze weight matrices |
| Parameter scaling | Efficient with more params | Train different sizes |
| Data scaling | Learns efficiently from data | Learning curves |
| Context scaling | Handles any context length | Already tested |
| Attention head scaling | Not fixed like transformers | Compare architectures |

## What Adrian Kosowski Said
> "There's one dimension which appears to be fixed... the size of the attention head vector dimension in a transformer... all concepts have to be mapped into a vector space of about 1,000 dimensions."

> "In BDH this can scale freely."

In [ ]:
# Setup
!pip install torch matplotlib numpy scipy networkx tqdm -q

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import networkx as nx
from tqdm.auto import tqdm
from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
# Clone repo if needed
import os
if not os.path.exists('bdh.py'):
    !git clone https://github.com/newsbubbles/bdh.git temp_bdh
    !cp temp_bdh/bdh.py .
    !cp temp_bdh/hierarchical_bdh.py .
    !rm -rf temp_bdh

from bdh import BDH, BDHConfig
from hierarchical_bdh import HierarchicalBDH, HierarchicalBDHConfig
print('Models loaded!')

---
## Test 1: Scale-Free Network Structure

BDH claims to have "scale-free" network structure like biological neural networks.
Scale-free networks have power-law degree distributions: P(k) ∝ k^(-γ)

Let's analyze the learned weight matrices as adjacency matrices.

In [ ]:
def analyze_weight_distribution(model):
    """
    Analyze weight matrices as if they were adjacency matrices.
    Check for power-law (scale-free) properties.
    """
    results = {}
    
    for name, param in model.named_parameters():
        if 'encoder' in name or 'decoder' in name:
            W = param.detach().cpu().numpy()
            
            # Treat absolute weights as connection strengths
            W_abs = np.abs(W)
            
            # Compute "degree" - sum of connection strengths per neuron
            # (treating rows as "from" and cols as "to")
            out_degree = W_abs.sum(axis=1)  # sum over columns
            in_degree = W_abs.sum(axis=0)   # sum over rows
            
            # Also look at number of "strong" connections
            threshold = np.percentile(W_abs, 90)  # top 10% are "connections"
            binary_W = (W_abs > threshold).astype(float)
            discrete_out = binary_W.sum(axis=1)
            discrete_in = binary_W.sum(axis=0)
            
            results[name] = {
                'shape': W.shape,
                'out_degree': out_degree,
                'in_degree': in_degree,
                'discrete_out': discrete_out,
                'discrete_in': discrete_in,
                'weight_distribution': W.flatten(),
            }
    
    return results

In [ ]:
def fit_power_law(data, min_val=None):
    """
    Fit power law P(x) ∝ x^(-alpha) to data.
    Returns alpha and goodness of fit.
    """
    data = data[data > 0]  # Remove zeros
    if len(data) < 10:
        return None, None, None
    
    if min_val is None:
        min_val = np.percentile(data, 10)
    
    data = data[data >= min_val]
    if len(data) < 10:
        return None, None, None
    
    # MLE for power law exponent
    # alpha = 1 + n / sum(ln(x/xmin))
    n = len(data)
    alpha = 1 + n / np.sum(np.log(data / min_val))
    
    # Kolmogorov-Smirnov test for goodness of fit
    # Compare empirical CDF to theoretical power law CDF
    sorted_data = np.sort(data)
    empirical_cdf = np.arange(1, n + 1) / n
    theoretical_cdf = 1 - (min_val / sorted_data) ** (alpha - 1)
    ks_stat = np.max(np.abs(empirical_cdf - theoretical_cdf))
    
    return alpha, ks_stat, min_val

In [ ]:
# Load trained model
checkpoint_path = 'checkpoints_hierarchical_small/best.pt'

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    state_dict = checkpoint.get('model_state_dict', checkpoint)
    
    if any('global_model' in k for k in state_dict.keys()):
        config = HierarchicalBDHConfig(**checkpoint.get('config', {}))
        model = HierarchicalBDH(config).to(device)
    else:
        config = BDHConfig(**checkpoint.get('config', {}))
        model = BDH(config).to(device)
    
    model.load_state_dict(state_dict)
    print('Loaded trained model')
else:
    config = BDHConfig(n_layer=4, n_embd=256, n_head=4, mlp_internal_dim_multiplier=64)
    model = BDH(config).to(device)
    print('Using untrained model (results less meaningful)')

# Analyze
weight_analysis = analyze_weight_distribution(model)
print(f'Analyzed {len(weight_analysis)} weight matrices')

In [ ]:
# Visualize degree distributions
n_matrices = len(weight_analysis)
fig, axes = plt.subplots(2, min(4, n_matrices), figsize=(16, 8))
if n_matrices < 4:
    axes = axes.reshape(2, -1)

power_law_results = {}

for idx, (name, data) in enumerate(list(weight_analysis.items())[:4]):
    # Continuous degree (weighted)
    ax = axes[0, idx]
    degrees = data['out_degree']
    ax.hist(degrees, bins=50, alpha=0.7, density=True)
    ax.set_xlabel('Weighted Out-Degree')
    ax.set_ylabel('Density')
    ax.set_title(f'{name.split(".")[-1]}\n{data["shape"]}')
    
    # Fit power law
    alpha, ks, xmin = fit_power_law(degrees)
    if alpha is not None:
        ax.text(0.95, 0.95, f'α={alpha:.2f}\nKS={ks:.3f}', 
                transform=ax.transAxes, ha='right', va='top',
                bbox=dict(boxstyle='round', facecolor='wheat'))
        power_law_results[name] = {'alpha': alpha, 'ks': ks}
    
    # Log-log plot for power law check
    ax = axes[1, idx]
    # Compute histogram for log-log
    hist, bin_edges = np.histogram(degrees[degrees > 0], bins=30)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    mask = hist > 0
    ax.scatter(np.log10(bin_centers[mask]), np.log10(hist[mask]), alpha=0.7)
    
    # Fit line
    if mask.sum() > 2:
        slope, intercept = np.polyfit(np.log10(bin_centers[mask]), np.log10(hist[mask]), 1)
        x_fit = np.linspace(np.log10(bin_centers[mask].min()), np.log10(bin_centers[mask].max()), 100)
        ax.plot(x_fit, slope * x_fit + intercept, 'r--', label=f'slope={slope:.2f}')
        ax.legend()
    
    ax.set_xlabel('log10(Degree)')
    ax.set_ylabel('log10(Count)')
    ax.set_title('Log-Log Plot (linear = power law)')

plt.tight_layout()
plt.savefig('scale_free_analysis.png', dpi=150)
plt.show()

In [ ]:
# Scale-free verdict
print('='*70)
print('SCALE-FREE NETWORK STRUCTURE ANALYSIS')
print('='*70)
print()
print('Power law exponents (α) for weight matrices:')
print('(Scale-free networks typically have 2 < α < 3)')
print()

scale_free_count = 0
for name, result in power_law_results.items():
    alpha = result['alpha']
    ks = result['ks']
    is_scale_free = 2 < alpha < 3 and ks < 0.1
    status = '✓' if is_scale_free else '~' if 1.5 < alpha < 4 else '✗'
    if is_scale_free:
        scale_free_count += 1
    print(f'  {name}: α={alpha:.2f}, KS={ks:.3f} {status}')

print()
if scale_free_count > len(power_law_results) // 2:
    print('✓ CLAIM SUPPORTED: Weight matrices show scale-free properties')
elif scale_free_count > 0:
    print('~ PARTIALLY SUPPORTED: Some matrices show scale-free properties')
else:
    print('✗ CLAIM NOT CLEARLY SUPPORTED: No strong power-law in weights')
    print('  (Note: Scale-free structure may emerge differently in BDH)')

---
## Test 2: Parameter Scaling Efficiency

How efficiently does BDH use additional parameters?
Compare loss vs parameter count for different model sizes.

In [ ]:
# Define model size configurations
model_configs = {
    'tiny': {'n_layer': 2, 'n_embd': 128, 'n_head': 2, 'mlp_internal_dim_multiplier': 32},
    'small': {'n_layer': 4, 'n_embd': 256, 'n_head': 4, 'mlp_internal_dim_multiplier': 64},
    'base': {'n_layer': 6, 'n_embd': 384, 'n_head': 6, 'mlp_internal_dim_multiplier': 96},
    'large': {'n_layer': 8, 'n_embd': 512, 'n_head': 8, 'mlp_internal_dim_multiplier': 128},
}

# Count parameters for each
param_counts = {}
for name, cfg in model_configs.items():
    model = BDH(BDHConfig(**cfg))
    n_params = sum(p.numel() for p in model.parameters())
    param_counts[name] = n_params
    print(f'{name}: {n_params:,} parameters')
    del model

In [ ]:
# Quick training to measure scaling
# (This cell takes a while - skip if you already have checkpoints)

RUN_SCALING_TEST = False  # Set to True to run

if RUN_SCALING_TEST:
    from datasets import load_dataset
    
    # Load data
    dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
    text = '\n'.join(dataset['text'])
    data = torch.tensor([b for b in text.encode('utf-8')], dtype=torch.long)
    
    def quick_train(config, data, steps=500, batch_size=32, block_size=256):
        model = BDH(BDHConfig(**config)).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
        
        losses = []
        for step in range(steps):
            # Random batch
            idx = torch.randint(0, len(data) - block_size, (batch_size,))
            x = torch.stack([data[i:i+block_size] for i in idx]).to(device)
            y = torch.stack([data[i+1:i+block_size+1] for i in idx]).to(device)
            
            logits = model(x)
            loss = F.cross_entropy(logits.view(-1, 256), y.view(-1))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
        
        return np.mean(losses[-50:])  # Final loss
    
    scaling_results = {}
    for name, cfg in tqdm(model_configs.items()):
        try:
            final_loss = quick_train(cfg, data)
            scaling_results[name] = {
                'params': param_counts[name],
                'loss': final_loss
            }
            print(f'{name}: loss={final_loss:.4f}')
        except RuntimeError as e:
            print(f'{name}: OOM')
else:
    print('Scaling test skipped. Set RUN_SCALING_TEST=True to run.')
    print('Using placeholder data for visualization...')
    # Placeholder based on typical results
    scaling_results = {
        'tiny': {'params': param_counts['tiny'], 'loss': 2.5},
        'small': {'params': param_counts['small'], 'loss': 1.8},
        'base': {'params': param_counts['base'], 'loss': 1.4},
        'large': {'params': param_counts['large'], 'loss': 1.2},
    }

In [ ]:
# Plot scaling curve
fig, ax = plt.subplots(figsize=(10, 6))

params = [scaling_results[k]['params'] for k in model_configs.keys() if k in scaling_results]
losses = [scaling_results[k]['loss'] for k in model_configs.keys() if k in scaling_results]
names = [k for k in model_configs.keys() if k in scaling_results]

ax.scatter(params, losses, s=100, zorder=5)
for i, name in enumerate(names):
    ax.annotate(name, (params[i], losses[i]), textcoords='offset points', xytext=(5,5))

# Fit power law: loss = a * params^(-b)
log_params = np.log(params)
log_losses = np.log(losses)
slope, intercept = np.polyfit(log_params, log_losses, 1)

x_fit = np.linspace(min(params), max(params), 100)
y_fit = np.exp(intercept) * x_fit ** slope
ax.plot(x_fit, y_fit, 'r--', label=f'Loss ∝ params^{slope:.3f}')

ax.set_xlabel('Parameters', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)
ax.set_title('BDH Scaling Law: Loss vs Parameters', fontsize=14)
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('parameter_scaling.png', dpi=150)
plt.show()

print(f'\nScaling exponent: {slope:.3f}')
print(f'(Chinchilla found ~-0.34 for transformers)')
if slope < -0.3:
    print('✓ BDH shows good parameter scaling efficiency')
elif slope < -0.2:
    print('~ BDH shows moderate scaling')
else:
    print('✗ BDH shows weak scaling with parameters')

---
## Test 3: Attention Head Dimension Flexibility

Transformers have fixed attention head dimension (~64-128).
BDH claims this can "scale freely". Let's test different dimensions.

In [ ]:
# In BDH, the "attention dimension" is controlled by mlp_internal_dim_multiplier
# Let's see how this affects performance

mlp_multipliers = [16, 32, 64, 128, 256]

print('BDH Latent Space Dimensions:')
print('(This is analogous to attention head dimension in transformers)')
print()

for mult in mlp_multipliers:
    config = BDHConfig(n_layer=4, n_embd=256, n_head=4, mlp_internal_dim_multiplier=mult)
    model = BDH(config)
    
    # Get actual latent dimension
    latent_dim = config.n_embd * mult // config.n_head
    n_params = sum(p.numel() for p in model.parameters())
    
    print(f'Multiplier {mult:3d}: Latent dim = {latent_dim:5d}, Params = {n_params:,}')
    del model

print()
print('For comparison, Transformer attention heads are typically 64-128 dims')
print('BDH can use much larger latent spaces (thousands of dims)')

In [ ]:
# Test if larger latent dimension helps
# (Quick test - increase steps for more reliable results)

RUN_LATENT_TEST = False  # Set to True to run

if RUN_LATENT_TEST:
    from datasets import load_dataset
    
    dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
    text = '\n'.join(dataset['text'])
    data = torch.tensor([b for b in text.encode('utf-8')], dtype=torch.long)
    
    latent_results = {}
    
    for mult in [32, 64, 128]:
        config = {'n_layer': 4, 'n_embd': 256, 'n_head': 4, 'mlp_internal_dim_multiplier': mult}
        
        try:
            model = BDH(BDHConfig(**config)).to(device)
            optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
            
            losses = []
            for step in tqdm(range(300), desc=f'mult={mult}'):
                idx = torch.randint(0, len(data) - 256, (32,))
                x = torch.stack([data[i:i+256] for i in idx]).to(device)
                y = torch.stack([data[i+1:i+257] for i in idx]).to(device)
                
                logits = model(x)
                loss = F.cross_entropy(logits.view(-1, 256), y.view(-1))
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                losses.append(loss.item())
            
            latent_results[mult] = np.mean(losses[-30:])
            del model
            torch.cuda.empty_cache()
        except RuntimeError:
            print(f'OOM at mult={mult}')
    
    print('\nResults:')
    for mult, loss in latent_results.items():
        print(f'  Multiplier {mult}: Loss = {loss:.4f}')
else:
    print('Latent dimension test skipped.')
    print('Set RUN_LATENT_TEST=True to run.')

---
## Test 4: Data Efficiency (Learning Curves)

How efficiently does BDH learn from data compared to parameter count?

In [ ]:
# If you have training logs, load and analyze them
# Otherwise, this shows how to analyze learning curves

print('='*70)
print('DATA EFFICIENCY ANALYSIS')
print('='*70)
print()
print('To properly test data efficiency, compare:')
print('1. Loss vs tokens seen (should follow power law)')
print('2. Optimal model size vs dataset size (Chinchilla scaling)')
print()
print('From your WikiText-2 training:')
print('- Started at loss ~3.2')
print('- Reached loss ~0.18 in 5000 steps')
print('- With batch_size=64, block_size=512')
print('- Thats ~160M tokens processed')
print()
print('This suggests BDH learns efficiently from data,')
print('achieving low perplexity with relatively few tokens.')

---
## Summary

In [ ]:
print('='*70)
print('BDH "SCALE-FREE" CLAIMS SUMMARY')
print('='*70)
print()

print('The term "scale-free" in BDH appears to mean multiple things:')
print()

print('1. SCALE-FREE NETWORK STRUCTURE')
print('   Claim: Power-law connectivity like biological networks')
print('   Finding: [Run analysis above for verdict]')
print()

print('2. FLEXIBLE LATENT DIMENSION')
print('   Claim: Not limited to ~1000 dims like transformer attention')
print('   Finding: ✓ BDH can use arbitrary latent dimensions')
print('            (mlp_internal_dim_multiplier controls this)')
print()

print('3. CONTEXT LENGTH SCALING')
print('   Claim: Handles unlimited context efficiently')
print('   Finding: [See unlimited context notebook]')
print()

print('4. PARAMETER SCALING')
print('   Claim: Efficient scaling with model size')
print('   Finding: [Run scaling test above for verdict]')
print()

print('='*70)
print('INTERPRETATION')
print('='*70)
print()
print('The BDH paper uses "scale-free" loosely. The strongest claim is:')
print('- BDHs latent dimension is not artificially constrained')
print('- Transformers fix attention head dim at ~64-128')
print('- BDH can use thousands of dimensions in its sparse space')
print()
print('This flexibility + sparsity may enable better concept representation.')